## Dependencies

In [1]:
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
model_path = '/kaggle/input/98-googleq-a-train-bert-base-glorot-uniform/model.h5'
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
TOKENIZER_PATH = BERT_PATH+'/assets/vocab.txt'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())
    # Map misspellings
    test[feature] = test[feature].apply(lambda x: map_misspellings(x))
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(TOKENIZER_PATH, True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

bert_layer = KerasLayer(BERT_PATH, trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

x = GlobalAveragePooling1D()(sequence_output)
x = Dropout(0.2)(x)
output = Dense(N_CLASS, kernel_initializer='glorot_uniform', activation="sigmoid", name="output")(x)

model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_masks[0][0]            

# Make predictions

In [8]:
Y_test = model.predict(X_test)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.939102,0.511940,0.340193,0.364054,0.399145,0.562076,0.579062,0.566970,0.715753,...,0.901795,0.898144,0.471815,0.968105,0.942777,0.830728,0.076493,0.041545,0.852259,0.920353
1,46,0.935530,0.481924,0.010182,0.706545,0.771724,0.917876,0.472946,0.410501,0.064689,...,0.729904,0.957823,0.623974,0.970226,0.982776,0.861475,0.959503,0.076961,0.066056,0.888155
2,70,0.932785,0.675606,0.060207,0.733469,0.858997,0.926114,0.543492,0.471201,0.110844,...,0.877412,0.930645,0.655842,0.980234,0.963207,0.857420,0.051727,0.048597,0.902822,0.921986
3,132,0.881668,0.410164,0.010456,0.633646,0.769069,0.918271,0.538701,0.450067,0.084530,...,0.691231,0.929162,0.660215,0.969139,0.983134,0.875315,0.915030,0.171010,0.269229,0.886171
4,200,0.923950,0.335921,0.123270,0.592022,0.683066,0.693801,0.548647,0.515692,0.081590,...,0.658049,0.831707,0.555079,0.962306,0.948797,0.769974,0.185092,0.130356,0.552618,0.924973


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.906723,0.542690,0.046082,0.664250,0.754104,0.845434,0.558738,0.457176,0.229990,...,0.792820,0.916199,0.649266,0.963517,0.969958,0.844637,0.565552,0.122758,0.479574,0.910288
std,2812.670060,0.042283,0.119956,0.075965,0.125928,0.121812,0.146927,0.052948,0.084104,0.195613,...,0.079918,0.029212,0.051651,0.013162,0.013650,0.044335,0.348692,0.064775,0.296096,0.022417
min,39.000000,0.723637,0.288498,0.004295,0.207090,0.239012,0.191330,0.420745,0.296922,0.006647,...,0.580575,0.811553,0.453137,0.863988,0.892040,0.692909,0.006288,0.008103,0.004045,0.798639
25%,2572.000000,0.880835,0.441635,0.009604,0.590678,0.716114,0.826430,0.521824,0.397319,0.085159,...,0.729192,0.899121,0.615858,0.957602,0.962832,0.817180,0.183973,0.075294,0.217746,0.896737
50%,5093.000000,0.913035,0.526300,0.014648,0.668304,0.776980,0.904310,0.551528,0.434248,0.158294,...,0.793046,0.919702,0.649319,0.965033,0.972989,0.849652,0.694869,0.118448,0.457606,0.911248
75%,7482.000000,0.939770,0.630759,0.041055,0.740433,0.824422,0.933743,0.591222,0.498912,0.334033,...,0.862779,0.937125,0.683967,0.971757,0.980105,0.876565,0.887039,0.165911,0.729404,0.925292
max,9640.000000,0.977875,0.872280,0.553783,0.978018,0.971457,0.982139,0.770949,0.732925,0.826076,...,0.943103,0.979733,0.778591,0.989866,0.991769,0.957213,0.985738,0.325872,0.987636,0.962638
